# JSON file generator
In the next code cell, change the parameter you want to loop for and it will creat as many .json file as you wish in current directory.

### Define Defult parameters

In [1]:
import numpy as np
#--------------------------------------#  1.4 Define Dirictiry and Load Model *(MOST IMPORTANT!)*

epoch = -1  #select previous checkpoint (-1 = don't use checkpoint)
#basedir = f"Test_v6.34_run1"

#--------------------------------------#  1.1 System Physical Parameters

dx,dy,dz = 50e-9,50e-9,20e-9  # discretization (m) for magnetic field calculation (50nm is said to be a little bit too large)

nx = 120          # size x    (magnetic cells)
ny = 80          # size y    (magnetic cells)

Ms = 140e3    # saturation magnetization (A/m)
B0 = 60e-3    # bias field (T)

geometry_type = 1 #Module Geometry ["1 # Flip magnets", "2 #Permanent magnets (training multiplier)", "3 #Permanent magnets"] 
Np = 4 #!# #number of actual probes

Ms_CoPt = 723e3 # saturation magnetization of the nanomagnets (A/m)
r0, dr, dm, z_off = 15, 4, 2, 10  # starting pos, period, magnet size, z distance
rx, ry = int((nx-2*r0)/dr), int((ny-2*r0)/dr+1) # build in calculation
damping_width = 10 # the damping boundary, in unit: cell 
sig_start_top=sig_start_bot=15 # signal generation line position, in unit: cell 

dt = 10e-12       #!#      # timestep (s)
timesteps = 1600 + 800  #!# # number of timesteps for wave propagation (nx*5 is the defult wave propergation time for nxny=50,dt=20e-12)

#--------------------------------------#  1.2 Training Parameters


optimizer_name = 'Adam'   #can be SGD,Adam,Adagrad,RMSprop,Adadelta ... (if changed, remember to change optimizer_params as well)
optimizer_params = {}     #{'momentum': 0.9,'weight_decay': 1e-4}  # extra parameter for the optimizer if needed, if not use '{}'.
loss_function = 'default' #"torch.nn.CrossEntropyLoss"  # loss function 
#apply_softmax = True      # `True` if the loss function only takes input in range [0,1]

learn_rate = 0.04   #!#       # Learning rate
epoch_max = 50    #!#         # number of training epoch

#--------------------------------------#  1.3 Define Training Data

f1 = 1e9      # source frequency (Hz) 
Bt = 1e-3     # excitation field amplitude (T)
time_duration = 100 #!# # `number of time step for each information`

# `how the frequency and amplitude changes`

A=5 
B=5.5 

Sig_f_1 = [A,B,A,B] + [0]
Sig_f_2 = [B,A,B,A] + [0]
Sig_f_3 = [A,A,B,B] + [0]
Sig_f_4 = [B,B,A,A] + [0]

A=1.0  #1.0e-3 T

Sig_Bt_1 = [A,A,A,A] + [0] 

f1_change_list=[Sig_f_1,Sig_f_2,Sig_f_3,Sig_f_4]
Bt_change_list=[Sig_Bt_1,Sig_Bt_1,Sig_Bt_1,Sig_Bt_1]
desired_out_list=[0,1,2,3] # the desired output for each signal 

#--------------------------------------#  Increase input signal size with separate noise

#--model training signals--
noise_std = Bt* 0.05
clone = 3 #!#
seed=42
batch_size = 3 #!#
#--sampeling/testing signals--
noise_std_test = Bt* 0.05
clone_test = 30 #!#
seed_test = 24

test_epochs=[0,24,36,49] #!#
validation_ratio = 1/3  # ratio of training data used for training the final regressor
probe_time_resolution = 2e-9 # in seconds
silent_pixel = 2

#--------------------------------------# parameters for the virtual probe method

detection_width = 5

#'equal_probes' or 'gradient_probes'
probe_reading_method_Train = 'gradient_probes' 
probe_reading_method_Test =  'equal_probes'

weighting_intersect = (0.0,0.2) # left value for normal label signal, right value for label ignore signal
weighting_sigma     = (1.5,1.5)
dist_cut_off  = (2.5,3.5) 

merge_ignore_with_last_probe=True #include the second probe intensity with ignore label vertual probes

#--------------------------------------#  1.5 Take note of the Model Detail
comments_for_this_training_and_testing= """
# Overall comment: 

try go back to multi-class task with the new pixel method
(double precision in use as well)

"""


### Define changing parameters and creat files
add as many loop as you like, remember to set each file a different name and basedir.

In [2]:
import json
param_names = ["basedir", "dx", "dy", "dz","nx","ny","Ms","B0","geometry_type","Np","Ms_CoPt", "r0", "dr", "dm", "z_off", "rx", "ry", 
            "damping_width", "sig_start_top", "sig_start_bot",
            "dt", "timesteps", "optimizer_name", "optimizer_params", "loss_function", "apply_softmax", "learn_rate", "epoch_max",
            "f1", "Bt", "time_duration", "f1_change_list", "Bt_change_list", "desired_out_list", 
            "noise_std","clone","seed","batch_size","noise_std_test","clone_test","seed_test","test_epochs",
            "validation_ratio","probe_time_resolution","silent_pixel","detection_width",
            "probe_reading_method_Train","probe_reading_method_Test","weighting_intersect","weighting_sigma","dist_cut_off",
            "merge_ignore_with_last_probe","comments_for_this_training_and_testing"]  

In [3]:
import os
if not os.path.isdir('json_files'):
    os.makedirs('json_files')

In [4]:
for method in ['equal','gradient']:
    for Bt_magnify in np.linspace(1.0,7.0,6): 
        for df in np.linspace(0.2,1.0,5):

            para_used_str=f"(method=[{method}],Bt={Bt_magnify:.1f}mT,df={df:.1f})"
            basedir = f"Compare_v6.35_method_large-df_{para_used_str}_"
            #-------------##-------------##-------------#
            
            if method=='gradient':
                probe_reading_method_Train = 'gradient_probes' 
            else :
                probe_reading_method_Train='equal_probes'

            #-------------

            f1 = 1e9      # source frequency (Hz) 

            A=5.0 
            B=5.0+ df 

            Sig_f_1 = [A,B,A,B] + [0]
            Sig_f_2 = [B,A,B,A] + [0]
            Sig_f_3 = [A,A,B,B] + [0]
            Sig_f_4 = [B,B,A,A] + [0] 

            A=1.0  #1.0e-3 T

            f1_change_list=[Sig_f_1,Sig_f_2,Sig_f_3,Sig_f_4]
            
            #-------------

            Bt = 1e-3
            A = Bt_magnify
            Sig_Bt_1 = [A,A,A,A] + [0] 
            Bt_change_list=[Sig_Bt_1,Sig_Bt_1,Sig_Bt_1,Sig_Bt_1]

            noise_std = Bt*Bt_magnify* 0.05
            noise_std_test = Bt*Bt_magnify* 0.05

            #-------------##-------------##-------------#
            parameters = {key: value for key, value in locals().items() if key in param_names and value is not None}
            with open(f"json_files/para_{para_used_str}.json", 'w') as fp:
                json.dump(parameters, fp)
            print(parameters)

{'dx': 5e-08, 'dy': 5e-08, 'dz': 2e-08, 'nx': 120, 'ny': 80, 'Ms': 140000.0, 'B0': 0.06, 'geometry_type': 1, 'Np': 4, 'Ms_CoPt': 723000.0, 'r0': 15, 'dr': 4, 'dm': 2, 'z_off': 10, 'rx': 22, 'ry': 13, 'damping_width': 10, 'sig_start_top': 15, 'sig_start_bot': 15, 'dt': 1e-11, 'timesteps': 2400, 'optimizer_name': 'Adam', 'optimizer_params': {}, 'loss_function': 'default', 'learn_rate': 0.04, 'epoch_max': 50, 'f1': 1000000000.0, 'Bt': 0.001, 'time_duration': 100, 'f1_change_list': [[5.0, 5.2, 5.0, 5.2, 0], [5.2, 5.0, 5.2, 5.0, 0], [5.0, 5.0, 5.2, 5.2, 0], [5.2, 5.2, 5.0, 5.0, 0]], 'Bt_change_list': [[1.0, 1.0, 1.0, 1.0, 0], [1.0, 1.0, 1.0, 1.0, 0], [1.0, 1.0, 1.0, 1.0, 0], [1.0, 1.0, 1.0, 1.0, 0]], 'desired_out_list': [0, 1, 2, 3], 'noise_std': 5e-05, 'clone': 3, 'seed': 42, 'batch_size': 3, 'noise_std_test': 5e-05, 'clone_test': 30, 'seed_test': 24, 'test_epochs': [0, 24, 36, 49], 'validation_ratio': 0.3333333333333333, 'probe_time_resolution': 2e-09, 'silent_pixel': 2, 'detection_width'